## 필요 라이브러리 호출

In [1]:
## 사용 라이브러리 호출
import pandas as pd
import numpy as np
import random 
from urllib.parse import quote, unquote
from datetime import timedelta

## 모델 사용 라이브러리 
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import trange
from sklearn.metrics import mean_squared_error, r2_score

## 모델 학습 결과 경로 설정 
import os
os.makedirs('./result', exist_ok=True)

## Cuda 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## 랜덤 시드 설정
def set_seed(seed_val):
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

# 시드 설정 
seed_val = 77
set_seed(seed_val)

cuda


## 사용 데이터 컬럼 선택

* tag name 이 많은 경우 tag name을 지정하는 것에 있어서 변수 설정이 다소 유연해짐
* tag name 은 순서대로 불러와짐

In [2]:
# tag name 출력 함수 
def show_column(URL):
    
    # Tag name 데이터 로드
    df = pd.read_csv(URL)
    
    # List 형식으로 변환
    df = df.values.reshape(-1)
    
    return df.tolist()

In [3]:
## tag name 출력 파라미터 설정
table = 'wind_elec_gen'

NAME_URL = f'http://127.0.0.1:5654/db/tql/datahub/api/v1/get_tag_names.tql?table={table}'

## tag name list 생성 
name = show_column(NAME_URL)

In [4]:
name

['h_6hrain',
 'h_6hsnow',
 'h_humidity',
 'h_rainprobability',
 'h_raintype',
 'h_seawave',
 'h_skystatus',
 'h_temperature',
 'h_winddirection',
 'h_windspeed',
 'hk1_1',
 'hk1_2',
 'hk1_3',
 'hk2_1',
 'hk2_2',
 'hk2_3',
 'hk2_4',
 'hk2_5',
 's_6hrain',
 's_6hsnow',
 's_humidity',
 's_rainprobability',
 's_raintype',
 's_seawave',
 's_skystatus',
 's_temperature',
 's_winddirection',
 's_windspeed',
 'ss_1',
 'ss_10',
 'ss_2',
 'ss_3',
 'ss_4',
 'ss_5',
 'ss_6',
 'ss_7',
 'ss_8',
 'ss_9']

## TAG Name format 변환 

* 위의 과정에서 Wind Elec Gen dataset의 모든 Tag Name 을 확인후 사용할 컬럼만 뽑아서 입력할 파라미터 형태로 변환
* 한경 제 1 발전소 관련 Tag Name 사용

In [5]:
# 원하는 tag name 설정
# 여기서 tag name 은 컬럼을 의미
tags = name[:13]

# 리스트의 각 항목을 작은따옴표로 감싸고, 쉼표로 구분
tags_ = ",".join(f"'{tag}'" for tag in tags)

# 사용 tag name 확인
print(tags_)

# 해당 값을 모델의 input shape로 설정 
print(len(tags))

'h_6hrain','h_6hsnow','h_humidity','h_rainprobability','h_raintype','h_seawave','h_skystatus','h_temperature','h_winddirection','h_windspeed','hk1_1','hk1_2','hk1_3'
13


## Wind Elec Gen Dataset 로드

* 한경 제 1 발전소 관련 Tag Name 들을 사용하여 데이터 로드

In [6]:
# 데이터 로드 함수
# 시간대를 맞추기 위해 3시간으로 resampling
# target 값으로 사용할 통합 발전량 계산후 추가 
def data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-rawdata.tql?table={table}&name={name}&start={start_time}&end={end_time}&timeformat={timeformat}')

    # 같은 시간대 별 데이터로 전환
    df = df.pivot_table(index='TIME', columns='NAME', values='VALUE', aggfunc='first').reset_index()

    # 'TIME' 열을 datetime 형식으로 변환
    df['TIME'] = pd.to_datetime(df['TIME'])

    # time index 설정
    df.set_index('TIME', inplace=True)

    # 3시간 간격으로 reampling 
    df = df.resample('3H').mean()

    # 결측값 제거
    df = df.dropna()

    # 발전량 병합
    # 한경 발전소의 1, 2, 3 발전량을 합침
    df.loc[:, 'hk1_total'] = df.iloc[:, 10:].sum(axis=1)
    
    return df

In [7]:
# 데이터 시간 로드 함수
def time_data_load(table, name, start_time, end_time, timeformat):
    
    # 데이터 로드 
    df = data_load(table, name, start_time, end_time, timeformat)
    
    # total 최대, 최소 값 계산
    max = df.iloc[:,-1:].max()
    min = df.iloc[:,-1:].min()
    
    # 타임 인덱스만 남김
    df = df.iloc[:,:0]
    
    return df, min, max

In [8]:
# 시간 변환 함수
# 시간 추가하려면 해당 과정 필요
# window_size : 데이터를 묶어서 수집되는 주기 
# step_size : 데이터의 간격 
def add_time(time_df, start_time, batch_size, window_size, step_size):
    
    # 몇개의 데이터를 로드해야 되는지 계산
    time = (batch_size * step_size)+ window_size - step_size - 1
    
    # 현재 시간의 인덱스 번호를 확인
    # 없는 경우는 맨처음 시간이 없는 경우이기 때문에 맨처음 인덱스로 지정함 
    try:
        index_now = time_df.index.get_loc(start_time)
    except KeyError:
        index_now = 0
    
    # 현재 시간 기준 배치 데이터의 마지막 시간 설정 
    end_time_ = str(time_df.index[index_now + time] + timedelta(hours=1))
    
    # 다음 시작 시간의 인덱스 번호 설정
    index_next = index_now + time - abs(window_size - step_size - 1)
    
    # 다음 시작 시간 설정
    next_start_time_ = str(time_df.index[index_next])
    
    # URL 인코딩
    start_time_ = quote(start_time)
    end_time_ = quote(end_time_)
    next_start_time_ = quote(next_start_time_)
    
    return start_time_, end_time_, next_start_time_, index_next

In [9]:
# 선택한 tag name 별 최대, 최소값 계산 함수
def set_minmax_value(table, name, start_time_train, end_time_train):
    
    # URL 인코딩
    start = quote(start_time_train)
    end = quote(end_time_train)
    
    # min max 데이터 로드
    df_ = pd.read_csv(f'http://127.0.0.1:5654/db/tql/datahub/api/v1/select-scale.tql?table={table}&name={name}&start={start}&end={end}')
    
    ## Min , Max values 설정 
    Min = df_.iloc[:,1:-1].T
    Max = df_.iloc[:,2:].T
    
    return Min, Max 

## 데이터 전처리
   * MinMaxScaler, window sliding 적용

In [10]:
def make_window(data, window_size, step_size):
    
    # 윈도우 슬라이딩 결과를 저장할 리스트
    windows = []
    targets = []
    
    # 슬라이딩 윈도우 적용
    for i in range(0, data.shape[0] - window_size + 1, step_size):
        
        window = data.iloc[i:i + window_size, :-1].values  # 마지막 컬럼 제외
        target_array = data.iloc[i:i + window_size, -1].values  # 마지막 컬럼이 target
            
        windows.append(window)
        targets.append(target_array)

    return windows, targets    

## Min-Max Scaling 설정

* Process 컨셉상 전체 데이터를 불러오지 않기 때문에 최대 최소값을 사용하는 방식의 Min-Max Scaler를 설정 

In [21]:
# MinMaxScaler 클래스 정의
class MinMaxScaler_custom:
    def __init__(self):
        self.min_ = None
        self.max_ = None

    # 설정 파라미터 기반 scale 값 설정 
    def transform(self, X, min_values, max_values):
        X = np.array(X)
        self.min_ = np.array(min_values)
        self.max_ = np.array(max_values)
        
        if self.min_ is None or self.max_ is None:
            raise ValueError("Min and Max values are not set.")
        
        # scale 값이 0이되는 것을 막기 위해 1e-6 을 더해줌 
        scale = (self.max_ - self.min_) + 1e-6
        if np.any(scale == 0):
            raise ValueError("Min and Max values are the same, resulting in a scale of 0.")
        
        return (X - self.min_) / scale
    
    # 계산한 scale 값 기준 데이터 정규화 
    def fit_transform(self, X, min_values, max_values):
        """Set parameters and then transform X"""
        return self.transform(X, min_values, max_values)

    # 정규화된 데이터를 원래 값으로 되돌림
    def inverse_transform(self, X_scaled):
        """Inverse the transformation and return original values"""
        if self.min_ is None or self.max_ is None:
            raise ValueError("Min and Max values are not set.")
        
        X_scaled = np.array(X_scaled)
        scale = (self.max_ - self.min_) + 1e-6
        
        return X_scaled * scale + self.min_

## 학습 모델 설정 

* LSTM AE 기본 모델 사용

In [12]:
# LSTM Autoencoder 클래스 정의
class LSTMAutoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(LSTMAutoencoder, self).__init__()
        
        # 인코더 LSTM
        self.encoder_lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.encoder_fc = nn.Linear(hidden_dim, int(hidden_dim/2))
        
        # 디코더 LSTM
        self.decoder_fc = nn.Linear(int(hidden_dim/2), hidden_dim)
        self.decoder_lstm = nn.LSTM(hidden_dim, input_dim, num_layers, batch_first=True)
        
        # FC Layer 설정
        self.fc1 = nn.Linear(input_dim,6)
        self.fc2 = nn.Linear(6,1)

    def forward(self, x):
        # 인코더 부분
        _, (h, _) = self.encoder_lstm(x)
        latent = self.encoder_fc(h[-1])
        
        # 디코더 부분
        hidden = self.decoder_fc(latent).unsqueeze(0).repeat(x.size(1), 1, 1).permute(1, 0, 2)
        output, _ = self.decoder_lstm(hidden)
        
        # FC Layer
        output = self.fc1(output)
        output = self.fc2(output)
        return output

In [13]:
# 모델 설정 파라미터

# 입력 데이터 컬럼 수
# PCA 한값 
# print(list(train_dataloader)[0][0].shape) 에서 마지막 숫자 
input_dim = 13

# LSMT hidden state 크기
hidden_dim = 6

# layer 수
num_layers = 2

# 학습률 
learning_rate = 0.001

# 모델 초기화
model = LSTMAutoencoder(input_dim, hidden_dim, num_layers).to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 모델 구조 확인
print(model)

LSTMAutoencoder(
  (encoder_lstm): LSTM(13, 6, num_layers=2, batch_first=True)
  (encoder_fc): Linear(in_features=6, out_features=3, bias=True)
  (decoder_fc): Linear(in_features=3, out_features=6, bias=True)
  (decoder_lstm): LSTM(6, 13, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=13, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=1, bias=True)
)


## 모델 학습 설정

* 학습 중 validation 데이터 기준 MSE 값이 제일 낮은 모델을 저장 

In [14]:
# 모델 학습 함수 설정
def train(table, name, timeformat, model, start_time_train, end_time_train, start_time_valid, end_time_valid, batch_size, window_size, step_size, epochs, Min, Max, min, max, scaler_data, scaler_target, time_df_train, time_df_valid):
    # 초기 train loss 설정
    train_loss = []

    # 베스트 loss 초기화 
    best_Loss=np.inf

    for epoch in epochs:
        
        # 학습 모드 설정 
        model.train()
        
        # 초기 loss 초기화
        running_loss = 0.0
        total_step = 0
        
        # 초기 시작 시간 설정
        start_time_ = start_time_train
        
        # 끝 시간 설정
        end_time_train = str(time_df_train.index[-1])

        # while 문을 통해 데이터 호출 
        while start_time_ < end_time_train:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_, end_time_, next_start_time_, index_next= add_time(time_df_train, start_time_, batch_size, window_size, step_size)
        
            # 데이터 로드 
            data = data_load(table, name, start_time_, end_time_, timeformat)

            # MinMax scaler 적용 
            data_scaled = scaler_data.fit_transform(data.iloc[:,:-1], Min, Max)
            target_scaled = scaler_target.fit_transform(data.iloc[:,-1:], min, max)
            
            # 데이터 프레임 + target 설정
            data_scaled = pd.DataFrame(data_scaled)
            data_scaled['target'] = target_scaled

            # window 설정 
            windows, targets = make_window(data_scaled, window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows) == batch_size:
                
                # 총 배치수 체크용  
                total_step = total_step + 1
                
                # 데이터를 numpy 배열로 변환
                input_data = np.array(windows)
                input_target = np.array(targets)
                
                # 데이터를 Tensor로 변환
                input_data = torch.tensor(input_data, dtype=torch.float32).to(device).float()
                input_target = torch.tensor(input_target, dtype=torch.float32).to(device).float()
                
                # 옵티마이저 최적화 
                optimizer.zero_grad()
                
                # 모델 입력
                outputs = model(input_data)
                
                # loss 계산
                loss = criterion(outputs.squeeze(), input_target)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                
                # 배치 리셋
                windows = []
                
            # 다음 시작 시간 설정    
            start_time_ = unquote(next_start_time_)
            
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next + (batch_size * step_size)+ window_size - step_size - 1 >= len(time_df_train):
                break
            
        train_loss.append(running_loss/total_step)
        print(f'\ntrain loss: {np.mean(train_loss)}')

        # Epoch 마다 validation을 진행해서 가장 좋은 성능을 보이는 모델을 저장 
        with torch.no_grad():
            
            model.eval()
            
            # 초기 Loss 설정 
            valid_loss = []
            
            # 초기 loss 초기화
            running_loss_v = 0.0
            total_step_v = 0
                
            # 초기 시작 시간 설정
            start_time_v = start_time_valid
            
            # 끝 시간 설정
            end_time_valid = str(time_df_valid.index[-1])
            
            # while 문을 통해 데이터 호출 
            while start_time_v < end_time_valid:
                
                # 배치 크기에 따라 데이터 로드 
                start_time_v, end_time_v, next_start_time_v, index_next_v = add_time(time_df_valid, start_time_v, batch_size, window_size, step_size)
                
                # 데이터 로드 
                data_v = data_load(table, name, start_time_v, end_time_v, timeformat)

                # MinMax scaler 적용 
                data_scaled_v = scaler_data.fit_transform(data_v.iloc[:,:-1], Min, Max)
                target_scaled_v = scaler_target.fit_transform(data_v.iloc[:,-1:], min, max)
                
                # 데이터 프레임 + target 설정
                data_scaled_v = pd.DataFrame(data_scaled_v)
                data_scaled_v['target'] = target_scaled_v

                # window 설정 
                windows_v, targets_v = make_window(data_scaled_v , window_size, step_size)
                
                # 로드한 데이터가 비어 있을 경우 출력 
                if len(data_v) == 0:
                    print("데이터가 없습니다.")
                
                # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
                if len(windows_v) == batch_size:
                    
                    # 총 배치수 체크용  
                    total_step_v = total_step_v + 1
                    
                    # 데이터를 numpy 배열로 변환
                    input_data_v = np.array(windows_v)
                    input_target_v = np.array(targets_v)

                    # 데이터를 Tensor로 변환
                    input_data_v = torch.tensor(input_data_v, dtype=torch.float32).to(device).float()
                    input_target_v = torch.tensor(input_target_v, dtype=torch.float32).to(device).float()
                    
                    # 모델 입력
                    outputs_v = model(input_data_v)
                    
                    # loss 계산
                    loss_v = criterion(outputs_v.squeeze(), input_target_v)
                    running_loss_v += loss_v.item()
                    
                    # 배치 리셋
                    windows_v = []
                
                valid_loss.append(running_loss_v/total_step_v)
                    
                # 다음 시작 시간 설정    
                start_time_v = unquote(next_start_time_v)
                
                # 마지막 시간을 넘어 가져오는 것을 방지
                if index_next_v + (batch_size * step_size)+ window_size - step_size - 1 >= len(time_df_valid):
                    break
                
            if best_Loss > np.mean(valid_loss):
                best_Loss = np.mean(valid_loss)
                torch.save(model, f'./result/Wind_Elec_Gen_New_Batch.pt')
                print('모델 저장')
            epochs.set_postfix_str(f"epoch = {epoch}, best_Loss = {best_Loss}")  
            
    return model

In [15]:
########################################### 공통 파라미터 설정 ################################################
# tag table 이름 설정
table = 'wind_elec_gen'
# tag name 설정
name = quote(tags_, safe=":/")
# 시간 포멧 설정 
timeformat = 'default'
# 배치 사이즈 설정
batch_size = 16
# window size 설정
window_size = 8
# step size 설정 -> window size 보다 크거나 같으면 안됨 
step_size = 1
# epoch 설정
epochs = trange(100, desc='training')
# Min-Max scaler 설정 
scaler_data = MinMaxScaler_custom()
scaler_target = MinMaxScaler_custom()

########################################### 학습 파라미터 설정 ################################################
# 학습 시작 시간 설정
start_time_train = '2014-01-01 06:00:00'
# 학습 끝 시간 설정
end_time_train = '2016-01-01 00:00:00'
# 최대, 최소값 설정 
Min, Max = set_minmax_value(table, name, start_time_train, end_time_train)
# 학습 시간 리스트 로드 
time_df_train, min, max = time_data_load(table, name, quote(start_time_train), quote(end_time_train), timeformat)

########################################### 검증 파라미터 설정 ################################################
# 검증 시작 시간 설정
start_time_valid = '2016-01-01 00:00:00'
# 검증 끝 시간 설정
end_time_valid = '2016-04-03 03:00:00'
# valid 시간 리스트 로드 
time_df_valid,_ ,_= time_data_load(table, name, quote(start_time_valid), quote(end_time_valid), timeformat)

# 학습 및 검증 진행
train(table, name, timeformat, model, start_time_train, end_time_train, start_time_valid, end_time_valid, batch_size, window_size, step_size, epochs, Min, Max, min, max, scaler_data, scaler_target, time_df_train, time_df_valid)

training:   0%|          | 0/100 [00:00<?, ?it/s]


train loss: 0.11514101648686077
모델 저장

train loss: 0.07896050558449769
모델 저장

train loss: 0.06301201689964091
모델 저장

train loss: 0.05452204142976409
모델 저장

train loss: 0.04923053199026813
모델 저장

train loss: 0.045606008223237716
모델 저장

train loss: 0.04290481614524182
모델 저장

train loss: 0.0397763626229354
모델 저장

train loss: 0.03685090704934926
모델 저장

train loss: 0.03438261714820183
모델 저장

train loss: 0.032317210486170346
모델 저장

train loss: 0.030570395706466486
모델 저장

train loss: 0.029075276641404628

train loss: 0.027780504439183087

train loss: 0.026647957872711493

train loss: 0.025649029059562397
모델 저장

train loss: 0.0247614738432819
모델 저장

train loss: 0.023967733518541482
모델 저장

train loss: 0.023253237776531793
모델 저장

train loss: 0.022606169916609943
모델 저장

train loss: 0.022017034603089944
모델 저장

train loss: 0.021478027314224046
모델 저장

train loss: 0.020982747543652404
모델 저장

train loss: 0.020525906877986805
모델 저장

train loss: 0.020103048387544578
모델 저장

train loss: 0.019710365986605

LSTMAutoencoder(
  (encoder_lstm): LSTM(13, 6, num_layers=2, batch_first=True)
  (encoder_fc): Linear(in_features=6, out_features=3, bias=True)
  (decoder_fc): Linear(in_features=3, out_features=6, bias=True)
  (decoder_lstm): LSTM(6, 13, num_layers=2, batch_first=True)
  (fc1): Linear(in_features=13, out_features=6, bias=True)
  (fc2): Linear(in_features=6, out_features=1, bias=True)
)

## 모델 테스트

In [16]:
# 베스트 모델 로드
model_ = torch.load(f'./result/Wind_Elec_Gen_New_Batch.pt')

In [22]:
def test(table, name, timeformat, model, start_time_test, end_time_test, batch_size, window_size, step_size, Min, Max, min, max, scaler_data, scaler_target, time_df_test):
    with torch.no_grad():
                
        model.eval()
        
        # 초기 설정 
        output_test = []
        output_target = []
            
        # 초기 시작 시간 설정
        start_time_t = start_time_test
        
        # 끝 시간 설정
        end_time_test = str(time_df_test.index[-1])
        
        # while 문을 통해 데이터 호출 
        while start_time_t < end_time_test:
            
            # 배치 크기에 따라 데이터 로드 
            start_time_t, end_time_t, next_start_time_t, index_next_t = add_time(time_df_test, start_time_t, batch_size, window_size, step_size)
            
            # 데이터 로드 
            data_t = data_load(table, name, start_time_t, end_time_t, timeformat)

            # MinMax scaler 적용 
            data_scaled_t = scaler_data.fit_transform(data_t.iloc[:,:-1], Min, Max)
            target_scaled_t = scaler_target.fit_transform(data_t.iloc[:,-1:], min, max)
            
            # 데이터 프레임 + target 설정
            data_scaled_t = pd.DataFrame(data_scaled_t)
            data_scaled_t['target'] = target_scaled_t

            # window 설정 
            windows_t, targets_t = make_window(data_scaled_t , window_size, step_size)
            
            # 로드한 데이터가 비어 있을 경우 출력 
            if len(data_t) == 0:
                print("데이터가 없습니다.")
            
            # 배치 사이즈 만큼 데이터가 쌓이면 다음 배치로 이동
            if len(windows_t) == batch_size:
                
                # 데이터를 numpy 배열로 변환
                input_data_t = np.array(windows_t)
                input_target_t = np.array(targets_t)

                # 데이터를 Tensor로 변환
                input_data_t = torch.tensor(input_data_t, dtype=torch.float32).to(device).float()
                input_target_t = torch.tensor(input_target_t, dtype=torch.float32).to(device).float()
                
                # 모델 입력
                outputs_t = model(input_data_t)
                
                output_test.append(outputs_t)
                output_target.append(input_target_t)
                
                # 배치 리셋
                windows_t = []

            # 다음 시작 시간 설정    
            start_time_t = unquote(next_start_time_t)
            
            # 마지막 시간을 넘어 가져오는 것을 방지
            if index_next_t + (batch_size * step_size)+ window_size - step_size - 1 > len(time_df_test):
                break
            
        # 텐서들을 하나로 합치기
        combined_tensor = torch.cat(output_test, dim=0)
        combined_target = torch.cat(output_target, dim=0)

        # window의 마지막 값만 사용하기
        last_values = combined_tensor[:, -1, :].cpu()
        last_target = combined_target.unsqueeze(2)[:, -1, :].cpu()

        # 데이터 프레임 생성
        final_scaled = pd.DataFrame(last_target, columns=['real'])
        final_scaled['pred'] = last_values

        # MinMaxScaler 역변환 
        last_values_ = scaler_target.inverse_transform(last_values)
        last_target_ = scaler_target.inverse_transform(last_target)

        # 데이터 프레임 생성
        final= pd.DataFrame(last_target_, columns=['real'])
        final['pred'] = last_values_
        
        return final_scaled, final 

In [25]:
########################################### 테스트 파라미터 설정 ################################################
# 검증 시작 시간 설정
start_time_test = '2016-01-01 06:00:00'
# 검증 끝 시간 설정
end_time_test = '2017-12-10 06:00:00'
# test 시간 리스트 로드 
time_df_test,_ ,_ = time_data_load(table, name, quote(start_time_test), quote(end_time_test), timeformat)

final_scaled, final = test(table, name, timeformat, model_, start_time_test, end_time_test, batch_size, window_size, step_size, Min, Max, min, max, scaler_data, scaler_target, time_df_test)

In [26]:
# MSE 계산
mse = mean_squared_error(final_scaled['real'].values, final_scaled['pred'].values)
print("Mean Squared Error:", mse)

# R² 스코어 계산
r2 = r2_score(final['real'].values, final['pred'].values)
print("R² Score:", r2)

Mean Squared Error: 0.0038355803
R² Score: 0.9595717178958093
